In [30]:
import networkx as nx
import gurobipy as gp

# Create a directed graph
G = nx.Graph()

# Define the edges representing relationships between people
G.add_edges_from ([
    ("Ashley", "Christopher"), ("Ashley", "Emily"), ("Ashley", "Joshua"),
    ("Bart", "Lisa"), ("Bart", "Matthew"), ("Christopher", "Andrew"),
    ("Emily", "Joshua"), ("Jacob", "Christopher"), ("Jessica", "Ashley"),
    ("JorEl", "Zod"), ("KalEl", "JorEl"), ("Kyle", "Lex"), ("Kyle", "Zod"),
    ("Lisa", "Marge"), ("Matthew", "Lisa"), ("Michael", "Christopher"),
    ("Michael", "Joshua"), ("Michael", "Jessica"), ("Samantha", "Matthew"),
    ("Samantha", "Tyler"), ("Sarah", "Andrew"), ("Sarah", "Christopher"),
    ("Sarah", "Emily"), ("Tyler", "Kyle"), ("Stuart", "Jacob")
])

# Initialize the Gurobi optimization model
m = gp.Model()

# Define binary decision variables for each edge in the graph
x = gp.tupledict()
# Since this is an undirected graph we need to be able to go in both directions
for (v1, v2) in G.edges:
    x[v1, v2] = m.addVar(vtype="B", name=f"x_{v1}_{v2}")
    x[v2, v1] = m.addVar(vtype="B", name=f"x_{v2}_{v1}")
    

# adding flow conservation constraints
for v in G.nodes:
    if v not in ["Ashley", "Zod"]:  # Replace with actual source and sink nodes if needed
        # Sum of incoming flows should equal sum of outgoing flows
        expr1 = gp.quicksum(x[i, v] for i in G.neighbors(v) if (i, v) in x or (v, i) in x)# Incoming flow
        expr2 = gp.quicksum(x[v, i] for i in G.neighbors(v) if (v, i) in x or (i, v) in x) # Outgoing flow
        
        m.addLConstr(expr1 - expr2 == 0)
        
# Function to compute shortest path distance
def get_shortest_path(source, target):
    try:
        distance = nx.shortest_path_length(G, source=source, target=target)
        path = nx.shortest_path(G, source=source, target=target)
        return f"Shortest path from {source} to {target}: {path} (Distance: {distance})"
    except nx.NetworkXNoPath:
        return f"No path exists between {source} and {target}"
    except nx.NodeNotFound:
        return f"These are not found in the same friend group."

# Promt the user for input
start = input("Choose start position: ")
end = input("Choose end point: ")


# Print the shortest path distance or notify if nodes are not connected
print(get_shortest_path(start, end))

Choose start position:  Marge
Choose end point:  KalEl


Shortest path from Marge to KalEl: ['Marge', 'Lisa', 'Matthew', 'Samantha', 'Tyler', 'Kyle', 'Zod', 'JorEl', 'KalEl'] (Distance: 8)
